In [6]:
import time
import math
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F
import zipfile

import sys
sys.path.append("..")
import d2lzh_pytorch as d2l
# device = torch.device('cpu')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def load_data_jay_lyrics():
    """加载歌词数据集"""
    with zipfile.ZipFile('../data/jaychou/jaychou_lyrics.txt.zip') as zin:
        with zin.open('jaychou_lyrics.txt') as f:
            corpus_chars = f.read().decode('utf-8')                             # corpus_chars 以字符形式读入数据
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')           # 将其中的换行符换为空格
    corpus_chars = corpus_chars[0:10000]                                        # 取前10000个字符作为训练的数据
    idx_to_char = list(set(corpus_chars))                                       # idx_to_char 按照字典顺序排列的汉字列表 如：{'去', '化', ' ', '和', '要',}
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])       # char_to_idx "汉字": "对应的索引" 形式的字典 如：{'去': 0, '化': 1, ' ': 2, '和': 3, '要': 4, '这': 5}
    vocab_size = len(char_to_idx)                                               # vocab_size 词典中不同字符的个数
    corpus_indices = [char_to_idx[char] for char in corpus_chars]               # corpus_indices 歌词顺序的每个字符对应字典中的索引
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = load_data_jay_lyrics()

### 初始化模型参数

In [7]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
print('will use', device)

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)
    def _three():
        return (_one((num_inputs, num_hiddens)),
                _one((num_hiddens, num_hiddens)),
                torch.nn.Parameter(torch.zeros(num_hiddens, device=device, dtype=torch.float32), requires_grad=True))

    W_xz, W_hz, b_z = _three()  # 更新门参数
    W_xr, W_hr, b_r = _three()  # 重置门参数
    W_xh, W_hh, b_h = _three()  # 候选隐藏状态参数

    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, dtype=torch.float32), requires_grad=True)
    return nn.ParameterList([W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q])

will use cuda


### 定义模型

In [8]:
def init_gru_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

def gru(inputs, state, params):
    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        Z = torch.sigmoid(torch.matmul(X, W_xz) + torch.matmul(H, W_hz) + b_z)
        R = torch.sigmoid(torch.matmul(X, W_xr) + torch.matmul(H, W_hr) + b_r)
        H_tilda = torch.tanh(torch.matmul(X, W_xh) + torch.matmul(R * H, W_hh) + b_h)
        H = Z * H + (1 - Z) * H_tilda
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)


### 训练

In [9]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']
d2l.train_and_predict_rnn(gru, get_params, init_gru_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, False, num_epochs, num_steps, lr,
                          clipping_theta, batch_size, pred_period, pred_len,
                          prefixes)


epoch 40, perplexity 151.343215, time 0.33 sec
 - 分开 我想你的让我不想想想想你你的可爱爱女人 坏坏的让我不想 你不你的让我不想 你想你我想想想你想你想你
 - 不分开 我想你的让我不想想想想你你的可爱爱女人 坏坏的让我不想 你不你的让我不想 你想你我想想想你想你想你
epoch 80, perplexity 32.600250, time 0.31 sec
 - 分开 一直我 别你的美笑 一彻后不觉 你不再再想 我不要再想 我不 我不 我不 我不 我不 我不 我不 
 - 不分开 不知不觉 我不要再想 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不
epoch 120, perplexity 5.975017, time 0.31 sec
 - 分开 一直走酒 你的完空 在小安动 一直风剧 我想就这样牵着你的手不放开 爱可不可以简简单单没有伤害 你
 - 不分开 说你我怕见 是一个酒 他来一直热粥 配上几斤的牛肉 我说店小了嵩山 学少 却想很了了吧? 我的在小
epoch 160, perplexity 1.833313, time 0.31 sec
 - 分开 一直到酒 你的完空前 开风透红抽无 它在什么我习惯 从小的话作  有有你对我 说你 从不会 为了是
 - 不分开 已经我回见你是一场悲剧 我想能这生你看你的手不放开 爱可不可以简简单单没有伤害 你 靠着我的肩膀 
